# 图学习初印象
### Part1 什么是图
- 图的两个基本元素：点、边
- 图是一种描述事物之间联系的语言
- 常见的图：社交网络、推荐系统、化学分子结构...

### Part2 什么是图学习
- 图学习: Graph Learning。深度学习中的一个子领域，处理的数据对象为图。
- 与一般深度学习的区别：能够方便地处理不规则数据（树、图），同时也可以处理规则数据（如图像）。

### Part3 图学习的应用
我们可以把图学习的应用分为节点级别任务、边级别任务、图级别任务。
课程中介绍了以下几种任务。
- 节点级别任务：金融诈骗检测（典型的节点分类）、自动驾驶中的3D点云目标检测
- 边级别任务：推荐系统（典型的边预测）
- 图级别任务：气味识别（典型的图分类）、发现“宇宙”

做一个小结：
![](https://ai-studio-static-online.cdn.bcebos.com/86ffa1763cdd492da8e04b3731c2156dec552e2fcb504f639a9fbd17ba23f949)


### Part4 图学习是怎么做的
- 图游走类算法：通过在图上的游走，获得多个节点序列，再利用 Skip Gram 模型训练得到节点表示（下节课内容）
- 图神经网络算法：端到端模型，利用消息传递机制实现。

### Part5 PGL 图学习库简介

- Github 链接：https://github.com/PaddlePaddle/PGL

- API文档： https://pgl.readthedocs.io/en/latest/

### Part6 课堂实践：熟悉 PGL 使用

#### 1. 环境安装

In [12]:
# 安装 PaddlePaddle 框架，由于 AIStudio 已配置相应环境，无需安装
# !pip install paddlepaddle==1.8.5

# 安装 PGL 学习库
!pip install pgl

#### 2. 使用 PGL 来创建一张图

假设我们有下面的这一张图，其中包含了10个节点以及14条边。
![](https://ai-studio-static-online.cdn.bcebos.com/3c5ea48a7c974c9784912375a70a8e5d731ec67af4004858bfdf8b1449c558d8)

我们的目的是，训练一个图模型，使得该图模型可以区分图上的黄色节点和绿色节点。我们可以使用以下代码来构图。

In [20]:
import numpy as np

import paddle
import paddle.nn as nn
import paddle.nn.functional as F
from paddle.optimizer import Adam
import pgl


def build_graph():
    # define the number of nodes; we can use number to represent every node
    num_node = 10
    # add edges, we represent all edges as a list of tuple (src, dst)
    edge_list = [(2, 0), (2, 1), (3, 1),(4, 0), (5, 0), 
             (6, 0), (6, 4), (6, 5), (7, 0), (7, 1),
             (7, 2), (7, 3), (8, 0), (9, 7)]

    # Each node can be represented by a d-dimensional feature vector, here for simple, the feature vectors are randomly generated.
    d = 16
    feature = np.random.randn(num_node, d).astype("float32")
    # each edge has it own weight
    edge_feature = np.random.randn(len(edge_list), 1).astype("float32")
    
    # create a graph
    g = pgl.Graph(edges = edge_list,
                  num_nodes = num_node,
                  node_feat = {'nfeat':feature}, 
                  edge_feat ={'efeat': edge_feature})

    return g
g = build_graph()

图创建完毕后，我们可以打印出图中的一些信息。

In [21]:
print('图中共计 %d 个节点' % g.num_nodes)
print('图中共计 %d 条边' % g.num_edges)

图中共计 10 个节点
图中共计 14 条边


#### 3. 定义图模型

创建一个简单的Graph Convolutional Network(GCN)网络。

In [22]:
class GCN(nn.Layer):
    """Implement of GCN
    """

    def __init__(self,
                 input_size,
                 num_class,
                 num_layers=2,
                 hidden_size=16,
                 **kwargs):
        super(GCN, self).__init__()
        self.num_class = num_class
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.gcns = nn.LayerList()
        for i in range(self.num_layers):
            if i == 0:
                self.gcns.append(
                    pgl.nn.GCNConv(
                        input_size,
                        self.hidden_size,
                        activation="relu",
                        norm=True))
            else:
                self.gcns.append(
                    pgl.nn.GCNConv(
                        self.hidden_size,
                        self.hidden_size,
                        activation="relu",
                        norm=True))
                
        self.output = nn.Linear(self.hidden_size, self.num_class)
    def forward(self, graph, feature):
        for m in self.gcns:
            feature = m(graph, feature)
        logits = self.output(feature)
        return logits

#### 4. 加标签

实现节点二分类，分别用0和1来表示两个类。

In [23]:
y = [0,1,1,1,0,0,0,1,0,1]
label = np.array(y, dtype="float32")

#### 5. 训练

In [24]:
g = g.tensor()
y = paddle.to_tensor(y)
gcn = GCN(16, 2)
criterion = paddle.nn.loss.CrossEntropyLoss()
optim = Adam(learning_rate=0.01, 
             parameters=gcn.parameters())
gcn.train()
for epoch in range(30):
    logits = gcn(g, g.node_feat['nfeat'])
    loss = criterion(logits, y)
    loss.backward()
    optim.step()
    optim.clear_grad()
    print("epoch: %s | loss: %.4f" % (epoch, loss.numpy()[0]))
    

/home/zzg/anaconda3/envs/py39_DL_cu112/lib/python3.9/site-packages/paddle/tensor/creation.py:130: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
W0104 14:16:04.042857 1655856 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 6.1, Driver API Version: 11.4, Runtime API Version: 10.2
W0104 14:16:04.057432 1655856 device_context.cc:465] device: 0, cuDNN Version: 8.1.


epoch: 0 | loss: 0.6382
epoch: 1 | loss: 0.5954
epoch: 2 | loss: 0.5662
epoch: 3 | loss: 0.5404
epoch: 4 | loss: 0.5219
epoch: 5 | loss: 0.5065
epoch: 6 | loss: 0.4946
epoch: 7 | loss: 0.4821
epoch: 8 | loss: 0.4692
epoch: 9 | loss: 0.4546
epoch: 10 | loss: 0.4391
epoch: 11 | loss: 0.4249
epoch: 12 | loss: 0.4132
epoch: 13 | loss: 0.4041
epoch: 14 | loss: 0.3975
epoch: 15 | loss: 0.3929
epoch: 16 | loss: 0.3902
epoch: 17 | loss: 0.3891
epoch: 18 | loss: 0.3890
epoch: 19 | loss: 0.3894
epoch: 20 | loss: 0.3899
epoch: 21 | loss: 0.3901
epoch: 22 | loss: 0.3899
epoch: 23 | loss: 0.3893
epoch: 24 | loss: 0.3885
epoch: 25 | loss: 0.3874
epoch: 26 | loss: 0.3863
epoch: 27 | loss: 0.3853
epoch: 28 | loss: 0.3844
epoch: 29 | loss: 0.3837
